In [3]:
# Import statements
import yfinance as yf
import pandas as pd
import os
import numpy as np
from datetime import date
from pathlib import Path
import pandas_ta as ta
from price_data_pull_yfinance import gethistoricalOHLC, saveHistStockData, loadHistDataFromDisk

from global_vars import dataPathToOHLC, pathToMasterDF, float16Cols, float32Cols

In [4]:
pd.set_option('display.max_columns', 500)

In [5]:
masterDf = pd.read_csv(pathToMasterDF);
masterDf.head()

,TICKER,FIRST_DATE_OHLC,LAST_DATE_OHLC,FILEPATH
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [6]:
masterDf = masterDf.iloc[0:0]

## Load the ticker symbols from the S&P 500 and download them via yahoo finance

The ticker symbols for the SnP500 are saved in the SnP500_constituents.csv. 

### Survivorship bias: 
Unfortunately, by using the latest version of the S&P500 we introduce a strong survivorship bias into our data sources. However, we are not able to optain more historical data. 
But, we are interested in the short term performance of stock. So that should be ok. 

In [7]:
from tqdm import tqdm
from time import sleep

In [9]:
SPY_ticker = pd.read_csv('../01_data/SnP500_constituents.csv');

In [11]:
tickerlist = SPY_ticker.Symbol.to_list();
# tickerlist

In [23]:
def downloadNSave(tickerlist):
    '''Download and save the tickerdata from yahoo finance and save them on disk.'''
    for ticker in tqdm(tickerlist):
        tickerDF = gethistoricalOHLC(ticker)
        saveHistStockData(ticker, tickerDF)
        sleep(0.1)

In [16]:
downloadNSave(tickerlist)

 13%|█▎        | 65/505 [00:38<04:00,  1.83it/s]

- BRK.B: No data found, symbol may be delisted
something went wrong with ticker BRK.B.


 15%|█▌        | 78/505 [00:45<03:17,  2.16it/s]

- BF.B: No data found for this date range, symbol may be delisted
something went wrong with ticker BF.B.


100%|██████████| 505/505 [06:05<00:00,  1.38it/s]


### VIX, treasury bonds and forex 
- The VIX is an indicator of market volatility -> extra feature (trades under the name '%5EVIX')
- Treasury bond yields can be seen as the risk free rate of return and are therefore an important compononent in the overall investment scenario
- Forex Data can be of interest for indication how strong the USD is compared to other countries. We will look at the Japanese Yen, as well as the Britisch Pound, and Canadian Dollar

#### Futher data ideas: 
- Unemployment rates
- expected GDP growth


In [31]:
data = [['VIX', '%5EVIX', 'Volatility Index'],
       ['IRX', '%5EIRX' , '13 Week Treasury Bill'],
       ['FVX', '%5EFVX' , 'Treasury Yield 5 Years'],
       ['TNX', '%5ETNX' , 'Treasury Yield 10 Years'],
       ['TYX', '%5ETYX' , 'Treasury Yield 30 Years']]
specialData = pd.DataFrame(data = data, columns = ['Symbol', 'URL_ticker', 'description'])
specialData.to_csv()


',Symbol,URL_ticker,description\r\n0,VIX,%5EVIX,Volatility Index\r\n1,IRX,%5EIRX,13 Week Treasury Bill\r\n2,FVX,%5EFVX,Treasury Yield 5 Years\r\n3,TNX,%5ETNX,Treasury Yield 10 Years\r\n4,TYX,%5ETYX,Treasury Yield 30 Years\r\n'

In [27]:
# VIX data
vixDF = gethistoricalOHLC('%5EVIX')
saveHistStockData('%5EVIX', vixDF)

True

In [32]:
# TRX data
vixDF = gethistoricalOHLC('%5EIRX')
saveHistStockData('%5EIRX', vixDF)

True

In [33]:
# FVX data
vixDF = gethistoricalOHLC('%5EFVX')
saveHistStockData('%5EFVX', vixDF)

True

In [34]:
# TNX data
vixDF = gethistoricalOHLC('%5ETNX')
saveHistStockData('%5ETNX', vixDF)

True

In [35]:
# TYX data
vixDF = gethistoricalOHLC('%5ETYX')
saveHistStockData('%5ETYX', vixDF)

True

# helper functions

In [47]:
def gethistoricalOHLC(ticker, start_date='1990-01-01', end_date=None):
    """finds a ticker and its representative OHLC data from yahoo finance.
    Returns a pandas dataframe"""

    if ticker is None:
        print("Ticker is empty!!!")

    if end_date == "" or end_date is None:
        end_date = str(date.today())

    # TODO #1 #create a checkup if that data is already in the database

    # instantiate the yf obj
    tickerObj = yf.Ticker(ticker)
    
    # get historical data
    df = tickerObj.history(start=start_date, end=end_date, interval="1d")
    
    if df.shape[0] > 0:

        try:
            df.reset_index(inplace=True)
            # cast column types
            float16TypeCast = [col for col in df.columns if col in float16Cols]
            float32TypeCast = [col for col in df.columns if col in float32Cols]
            df["Date"] = df["Date"].dt.date
            df[float16TypeCast] = df[float16TypeCast].astype("float16")
            df[float32TypeCast] = df[float32TypeCast].astype("float32")
        except:
            print(f'something went wrong with ticker {ticker}.')
    return df

- BRK.B: No data found, symbol may be delisted


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
